In [1]:
# https://pypi.org/project/jiwer/
#!pip install jiwer

In [21]:
import os
import wave
from vosk import Model, KaldiRecognizer, SetLogLevel
import json
import difflib
import math
import numpy as np
import requests
import jiwer

In [3]:
def make_file_splitted(side, original_file_name):
    
    # init ++
    temp_file_path = ''
    temp_file_name = 'temp'+('_R' if side else '_L')+'.wav'
    original_file_path = ''
    # init --
    
    # crop '.wav' & append postfix
    temp_file_name = original_file_name[:-4]+('_R' if side else '_L')+'.wav'

    os_cmd = 'ffmpeg -y -i '
    os_cmd += original_file_path
    os_cmd += original_file_name
    os_cmd += ' -ar 16000 -af "pan=mono|c0=F'
    os_cmd += 'R' if side else 'L'
    os_cmd += '" '
    os_cmd += temp_file_path
    os_cmd += temp_file_name

    try:
        os.system(os_cmd)
    except Exception as e:
        print('make_file_splitted error:',str(e))
    return os.path.isfile(temp_file_path + temp_file_name)

In [4]:
file_name = 'in_74992317889_2020-10-27-11-53-19rxtx'
temp_l = make_file_splitted(0,file_name+'.wav')
temp_r = make_file_splitted(1,file_name+'.wav')
temp_l, temp_r

(True, True)

In [5]:
def transcribe(side, file_name, source_id):

    # transcribation_date = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    # init ++
    temp_file_path = ''
    if source_id == 1:
        temp_file_name = file_name+('_R' if side else '_L')+'.wav'
    else:
        temp_file_name = file_name+('-in' if side else '-out')+'.wav'
    #model_path = '/home/alex/projects/vosk-api/python/example/model'
    ### adaptation ++
    model_path = '/home/alex/projects/vosk-api/python/example/vosk-model-ru-0.10'
    ### adaptation --
    phrases_list = []
    # init --

    # read file
    wf = wave.open(temp_file_path + temp_file_name, "rb")

    # read model
    model = Model(model_path)
    rec = KaldiRecognizer(model, wf.getframerate())
    ### adaptation ++
    #appended_words = '["да приорова шестнадцать корпус один хорошо ваш заказ отменен"]'
    #rec = KaldiRecognizer(model, wf.getframerate(), appended_words)
    ### adaptation --

    # recognizing
    #phrases_count = 0

    while True:

        conf_score = []

        data = wf.readframes(4000)
        if len(data) == 0:
            break

        if rec.AcceptWaveform(data):
            accept = json.loads(rec.Result())
            print(accept)
            if accept['text'] !='':

                accept_start	= str(accept['result'][0]['start'])
                accept_end   	= accept['result'][-1:][0]['end']
                accept_text		= str(accept['text'])

                for result_rec in accept['result']:
                    conf_score.append(float(result_rec['conf']))
                conf_mid = str(sum(conf_score)/len(conf_score))
                conf_score = []
                
                #self.save_result(accept_text, accept_start, accept_end, side, transcribation_date, conf_mid, linkedid, dst)
                phrases_list.append(accept_text)
                #phrases_count+=1

    #if phrases_count == 0:
        #self.save_result('', '0', '0', side, transcribation_date, 0, linkedid, dst)
    return phrases_list

In [6]:
def score(robot, human):
    scores = []
    for i in range(len(robot)):
        scores.append({
            'robot': robot[i],
            'human': human[i],
            'score': int(difflib.SequenceMatcher(None, robot[i], human[i]).ratio()*100)
        })
        print(scores[i]['score'], '#', scores[i]['robot'])
    print('average', np.average([i['score'] for i in scores]))
    print('median ', np.median([i['score'] for i in scores]))

In [24]:
def error(ground_truth, hypothesis):
    wer = jiwer.wer(ground_truth, hypothesis)
    mer = jiwer.mer(ground_truth, hypothesis)
    wil = jiwer.wil(ground_truth, hypothesis)

    # faster, because `compute_measures` only needs to perform the heavy lifting once:
    measures = jiwer.compute_measures(ground_truth, hypothesis)
    wer = measures['wer']
    mer = measures['mer']
    wil = measures['wil']

    print('wer', wer)
    print('mer', mer)
    print('wil', wil)

In [7]:
#master_talk_0 = transcribe(0, '1612424788.1439425', 2)
#master_talk_0

In [8]:
#master_talk_1 = transcribe(1, '1612424788.1439425', 2)
#master_talk_1

In [ ]:
human_text = []
# lo1615966037.2256002-out.wav	
human_text.append('але да да ну надо ремонтировать')
human_text.append('a')
human_text.append('a')
human_text.append('a')
human_text.append('a')
df.human_text = human_text

### vosk

In [9]:
vosk_phrases = transcribe(1, file_name, 1)

{'result': [{'conf': 1.0, 'end': 1.98, 'start': 0.061167, 'word': 'и'}], 'text': 'и'}
{'result': [{'conf': 0.53699, 'end': 4.47, 'start': 4.2, 'word': 'иди'}, {'conf': 0.278438, 'end': 4.829677, 'start': 4.47, 'word': 'нахуй'}, {'conf': 0.219468, 'end': 5.129524, 'start': 4.839676, 'word': 'ваша'}, {'conf': 0.32185, 'end': 5.456853, 'start': 5.129524, 'word': 'раса'}, {'conf': 0.914464, 'end': 5.938568, 'start': 5.456853, 'word': 'оператора'}, {'conf': 0.878963, 'end': 6.30055, 'start': 5.938568, 'word': 'оксана'}, {'conf': 0.464882, 'end': 6.93, 'start': 6.30055, 'word': 'здравствуйте'}], 'text': 'иди нахуй ваша раса оператора оксана здравствуйте'}
{'text': ''}
{'text': ''}
{'result': [{'conf': 1.0, 'end': 18.06, 'start': 17.730168, 'word': 'номер'}, {'conf': 0.621418, 'end': 18.719648, 'start': 18.06, 'word': 'заказан'}, {'conf': 1.0, 'end': 18.96, 'start': 18.719648, 'word': 'номер'}, {'conf': 1.0, 'end': 19.41, 'start': 18.96, 'word': 'телефона'}, {'conf': 1.0, 'end': 19.68, 'start

In [10]:
vosk_phrases

['и',
 'иди нахуй ваша раса оператора оксана здравствуйте',
 'номер заказан номер телефона могли бы назвать',
 'секунду',
 'приора стас корпус сайты ну хорошо ваш заказ от меня',
 'хорошо',
 'хорошо до свидания']

### sova

In [11]:
# https://github.com/sovaai/sova-asr

In [19]:
#filename = 'in_74992317889_2020-10-27-11-53-19rxtx_R.wav'
filename = 'in_74992317889_2020-10-27-11-53-19rxtx.wav'
url = "http://localhost:8888/asr/"
f1 = open(filename, 'rb')
response = requests.post(url,files={"audio_blob": f1}) #several files accepted maybe
print(response.text)

{"r": [{"response_audio_url": "/media/e7a15903-799a-4912-8d70-d2f840e0bec8.wav", "response_code": 0, "response": [{"text": "единая служба старается оператор крана здравствуйте ну вашей матери меня звонила это ну в общем будешь кататься своих нет это в общем заказ пока отмените состав я перезвоню там номер заказа номер телефона могли бы назвать номер девятьсот с семьсот сорок шесть девяносто три восемьдесят секунду это протокола звонила первосортная перевести что я перезвоню так хорошо спасибо все хорошо", "time": 31.842}]}]}


In [15]:
sova_phrases = [
    '',
    'единая служба старается оператор оксана здравствуйте',
    'номер заказа номер телефона могли бы назвать',
    'секунду',
    'да приор вас паспорт сабина хорошо ваш заказ от меня',
    'так хорошо',
    'все хорошо до свидания',
]

### comparing

In [16]:
human_phrases = [
    'и',
    'единая служба сервиса оператор оксана здравсвуйте',
    'номер заказа или номер телефона могли бы назвать',
    'секунду',
    'да приорова шестнадцать корпус один хорошо ваш заказ отменен',
    'да хорошо',
    'угу всего хорошего до свидания',
]

In [17]:
score(vosk_phrases, human_phrases)

100 # и
77 # иди нахуй ваша раса оператора оксана здравствуйте
94 # номер заказан номер телефона могли бы назвать
100 # секунду
75 # приора стас корпус сайты ну хорошо ваш заказ от меня
80 # хорошо
75 # хорошо до свидания
average 85.85714285714286
median  80.0


In [18]:
score(sova_phrases, human_phrases)

0 # 
87 # единая служба старается оператор оксана здравствуйте
95 # номер заказа номер телефона могли бы назвать
100 # секунду
67 # да приор вас паспорт сабина хорошо ваш заказ от меня
84 # так хорошо
84 # все хорошо до свидания
average 73.85714285714286
median  84.0


### error rate

In [25]:
error(human_phrases, vosk_phrases)

wer 0.5625
mer 0.5454545454545454
wil 0.765625


In [26]:
error(human_phrases, sova_phrases)

wer 0.4375
mer 0.4375
wil 0.6625000000000001
